In [1]:
import setGPU

import tensorflow as tf

import sys

sys.path.append('../scripts')

from data_reading import *
from autoencoder_architectures import *

setGPU: Setting GPU to: 1


/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vuu/anaconda3/envs/tfpy3_exp/lib/python3

In [2]:
data = public_datasets(False)
x_train_full, y_train_full, x_test, y_test = data.load_fashion_mnist()
print(x_train_full.shape, y_train_full.shape, x_test.shape, y_test.shape)

x_train = x_train_full[:-5000]
x_valid = x_train_full[-5000:]

y_train = y_train_full[:-5000]
y_valid = y_train_full[-5000:]

print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)
(55000, 28, 28) (55000,) (5000, 28, 28) (5000,)


In [3]:
def rounded_accuracy(y_true, y_pred):
    return tf.keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

In [4]:
np.random.seed(42)

stacked_encoder = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="selu"),
    tf.keras.layers.Dense(30, activation="selu"),
])
stacked_decoder = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100, activation="selu", input_shape=[30]),
    tf.keras.layers.Dense(28 * 28, activation="sigmoid"),
    tf.keras.layers.Reshape([28, 28])
])
stacked_ae = tf.keras.models.Sequential([stacked_encoder, stacked_decoder])

stacked_ae.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.SGD(lr=1.5), metrics=[rounded_accuracy])

history = stacked_ae.fit(x_train, x_train, epochs=20, validation_data=(x_valid, x_valid), verbose=2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 - 9s - loss: 0.3351 - rounded_accuracy: 0.8901 - val_loss: 0.3103 - val_rounded_accuracy: 0.9145
Epoch 2/20
55000/55000 - 8s - loss: 0.3043 - rounded_accuracy: 0.9166 - val_loss: 0.3012 - val_rounded_accuracy: 0.9226
Epoch 3/20
55000/55000 - 8s - loss: 0.2975 - rounded_accuracy: 0.9226 - val_loss: 0.2973 - val_rounded_accuracy: 0.9262
Epoch 4/20
55000/55000 - 8s - loss: 0.2930 - rounded_accuracy: 0.9269 - val_loss: 0.2922 - val_rounded_accuracy: 0.9289
Epoch 5/20
55000/55000 - 8s - loss: 0.2903 - rounded_accuracy: 0.9291 - val_loss: 0.2914 - val_rounded_accuracy: 0.9311
Epoch 6/20
55000/55000 - 8s - loss: 0.2885 - rounded_accuracy: 0.9308 - val_loss: 0.2900 - val_rounded_accuracy: 0.9317
Epoch 7/20
55000/55